In [1]:
from glob import glob
import yaml
import os
import json
import re
import hashlib
import shutil

In [2]:
# where to find game.yml
repo = '/home/p3nny/code4maus/src/lib/edu/*/game.yml'

In [3]:
# use md5 for compring dublicate assets 

def get_md5(filename):
    hash_md5= hashlib.md5()
    with open(filename, "rb") as f: 
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [4]:
# where to find assets and the new shared_assets folder

assets_repo = f'/home/p3nny/code4maus/src/lib/edu/*/assets/*'

# make shared assets folder by hand 
shared = '/home/p3nny/code4maus/src/lib/edu/shared_assets/'

In [5]:
def move_file(source, dest):
    os.rename(source, f'{shared}{dest}')

In [6]:
def imports_to_string(imports):
    js = ''
    for name, path in imports.items():
        js += (f'import {name} from \'{path}\';\n')
    return js

In [7]:
regex = re.compile('"asset": "(\w+)"')

In [8]:
def replacer(match):
    return f'"asset": {match.group(1)}'

In [9]:
# find duplicate assets and move them to shared_assets folder

maus_assets = {}
shared_assets = {}

for asset in glob(assets_repo):
    md5 = get_md5(asset)
    if md5 in maus_assets:
        first_occurence = maus_assets[md5]
        move_file(asset, os.path.basename(first_occurence))
        shared_assets[os.path.basename(asset)] = os.path.basename(first_occurence)
        try: 
            os.remove(first_occurence)
            shared_assets[os.path.basename(first_occurence)] = os.path.basename(first_occurence)
        except: 
            OSError
    else:
        maus_assets[md5] = asset

print(shared_assets)

{'herz.png': 'herz.png', '001.jpg': 'start.jpg', 'start.jpg': 'start.jpg', 'super.png': 'super.png', '003.jpg': 'start.jpg', 'gut_gemacht.png': 'gut_gemacht.png', 'L00.png': 'L00.png', 'steuerung.jpg': 'steuerung.jpg', 'frage.jpg': 'frage.jpg', 'speichern.jpg': 'speichern.jpg', 'stop.png': 'stop.png', 'operatoren.png': 'y_05.png', 'y_05.png': 'y_05.png', 'variablen_hand.png': 'variablen_hand.png', 'start.png': 'start.png'}


In [10]:
# convert game.yml to game.js and use imports to shared_assets

for filename in glob(repo):
    js = ''
    imports = {}
    game_id = os.path.basename(os.path.dirname(filename))        
    with open(filename, "r") as f:
        game = yaml.safe_load(f)
        for i, slide in enumerate(game['slides']):
            if 'asset' not in slide:
                continue
            asset = slide['asset']
            import_name = f'a{os.path.splitext(asset)[0]}'
            slide['asset'] = import_name
            if import_name not in imports:
                if asset in shared_assets:
                    imports[import_name] = f'../shared_assets/{shared_assets[asset]}'
                else:
                    imports[import_name] = f'./assets/{asset}'
        f.close()
    js += imports_to_string(imports)
    js += '\n'
    js += 'export default '
    js += json.dumps(game, indent=4, ensure_ascii=False)
    nicer_js = regex.sub(replacer, js)
    outpath = f'/home/p3nny/code4maus/src/lib/edu/{game_id}/game.js'
    with open(outpath, 'w', encoding='utf-8') as f:
        f.write(nicer_js)
        f.close()

In [11]:
# remove unused assets

for filename in glob(repo):
    game_id = os.path.basename(os.path.dirname(filename)) 
    all_assets = glob(f'/home/p3nny/code4maus/src/lib/edu/{game_id}/assets/*')
    assets = [os.path.basename(path) for path in all_assets]
    with open(filename, "r") as f:
        game = yaml.safe_load(f)
        for slide in game['slides']:
            if slide['asset'] in assets:
                assets.remove(slide['asset'])
        for asset in assets:
            os.remove(f'/home/p3nny/code4maus/src/lib/edu/{game_id}/assets/{asset}')

In [ ]:
# 

In [12]:
renames = {
'./01/assets/L01.png': './01/thumb.png',
'./02/assets/L02.png': './02/thumb.png',
'./03/assets/L03.png': './03/thumb.png',
'./04/assets/L04.png': './04/thumb.png',
'./05/assets/L05.png': './05/thumb.png',
'./04_a/assets/StadtLandEnte.png': './04_a/thumb.png',
'./04_b/assets/Mathefant.png': './04_b/thumb.png',
'./04_c/assets/L00.png': './04_c/thumb.png'
}

In [13]:
for oldpath, newpath in renames.items():
    oldpath_abs = os.path.join('/home/p3nny/code4maus/src/lib/edu/', oldpath[2:])
    newpath_abs = os.path.join('/home/p3nny/code4maus/src/lib/edu/', newpath[2:])
    shutil.copyfile(oldpath_abs, newpath_abs)

FileNotFoundError: [Errno 2] No such file or directory: '/home/p3nny/code4maus/src/lib/edu/04_c/assets/L00.png'

In [ ]:
with open('/home/p3nny/code4maus/src/lib/edu/index.js', 'r') as f:
    index = f.read()
    f.close()
    for oldpath, newpath in renames.items():
        index = index.replace(oldpath, newpath)
    with open('/home/p3nny/code4maus/src/lib/edu/index.js', 'w') as g:
        g.write(index)
        g.close()

## File conversion: .gif to .mp4 and .png to .jpg 